In [ ]:
import json
import requests
from google.cloud import storage

In [ ]:
# The ID of our GCP project
project_id = "CREATE PROJECT ID"
# The GCS bucket that contains notebook files
notebook_bucket_name = "CREATE bucket for notebook files"
# The GCS bucket that contains the original IRS files
irs_xml_bucket_name = "CREATE bucket for notebook files"

In [ ]:
# The list of XMLs is saved in a JSON file in our GCS
xml_url_list_file = "CREATE JSON file"

In [ ]:
storage_client = storage.Client(project=project_id)

In [ ]:
notebook_bucket = storage_client.bucket(notebook_bucket_name)

In [ ]:
xml_url_list_blob = notebook_bucket.blob(xml_url_list_file)

In [ ]:
xml_url_list_blob_content = xml_url_list_blob.download_as_text()

In [ ]:
xml_urls = json.loads(xml_url_list_blob_content)

In [ ]:
xml_urls.keys()

In [ ]:
upload_folder_prefix = "irs-form-990/original"

In [ ]:
upload_bucket = storage_client.bucket(irs_xml_bucket_name)

In [ ]:
# Download each file to local file system using requests library
# Then upload the files to GCS
# The whole process takes 30-40 minutes
# There must be at least 30 GB disk space available
for year in xml_urls.keys():
    urls = xml_urls[year]
    for url in urls:
        print(f"Downloading {url}")
        response = requests.get(url)
        file_name = url.split("/")[-1]
        with open(file_name, "wb") as f:
            f.write(response.content)
        upload_path = f"{upload_folder_prefix}/{year}/{file_name}"
        blob = upload_bucket.blob(upload_path)
        print(f"Uploading to {upload_path}")
        blob.upload_from_filename(file_name)
